1. IMPORT

In [ ]:
2. KEY POINTS USING MEDIA PIPE

3. EXTRACT KEY POINTS

4. SET UP FOLDERS FOR COLECTION

5. COLLECT KEY POINTS FOR TRININGS AND TESTING

6. PREPROCESS DATA

7. BUILD AND TRAIN LSTM NEURAL NET

8. MAKE PREDICITIONS 

9. SAVE WEIGHTS

10. EVAL USING CONFUSION MATRIX AND ACCURACY

11. TEST